# Online Brokerage - First draft of backend code

## Class Creation

In [69]:
import os

In [70]:
class User():
    def __init__(self, name, country, dob, password, balance, exchange):
        self.name = name
        self.country = country
        self.dob = dob
        self.__password = password
        self._portfolio = {}
        self.balance = balance
        self.exchange = exchange


    @property
    def country(self):
        return self._country


    @country.setter
    def country(self, country):
        """Setter method that validates and sets the user's country
        
        Reads in a list of countries and checks that the user's country is in this list
        If it is not, this implies that the user has provided a country that does not exist, and this will trigger an error
        """
        self._country_list = ReadCountries("countries.csv").countries
        if country in self._country_list:
            self._country = country

        else:
            raise ValueError("Error, invalid country selected")
        

    def buy(self, asset_name, quantity):
        """Function that allows the user to buy an asset
        
        First, sends a buy request to the exchange for validation and processing
        If the user's buy request is deemed to be valid, the transaction continues
        If the user already owns the asset, the quantity held associated with this asset is increased
        If the user does not own the asset, the asset is added to the portfolio, along with the quantity purchased
        A message is then displayed to tell the user if their transaction was valid or not and some information about the transaction
        """
        valid_request, asset = self.exchange.process_buy(asset_name, quantity, self.balance)
        
        if valid_request:
            if asset in self._portfolio:
                self._portfolio[asset] += quantity
            else:
                self._portfolio[asset] = quantity
            
            self.balance -= asset.price * quantity
            print(f"Successfully purchased {quantity} shares of {asset.name}")
            print(f"Current balance: {self.balance}")

        else:
            print(f"Falied to purchase {quantity} shares of {asset_name}")


    def sell(self, asset_name, quantity):
        """Function that allows the user to sell an asset
        
        First, checks that the asset exists in the portfolio and that a sufficient quantity of the asset exists
        If this is the case, a sell request is sent to the exchange for processing
        The quantity of the asset sold is removed from the user's portfolio
        The user's cash balance is reduced
        Finally, some messages are displayed to the user to give some details about the transaction
        """
        for asset in self._portfolio:
            if asset.name == asset_name:
                if self._portfolio[asset] >= quantity:
                    self.exchange.process_sale(asset_name, quantity)
                    self._portfolio[asset] -= quantity
                    self.balance += asset.price * quantity
                    print(f"Successfully sold {quantity} shares of {asset.name}")
                    print(f"Current balance: {self.balance}")
                    break

                else:
                    print("Error: insufficient quantity available")
                    print(f"Quantity requested: {quantity}")
                    print(f"Quantity available: {self._portfolio[asset]}")
                    break

        else:
            print(f"Error: {asset_name} not found in portfolio")
            print("Please ensure that you own the asset you wish to sell and that you have input the correct ticker symbol")


    def show_portfolio(self):
        print(f"{self.name}'s Portfolio")
        print("Name", "\t", "Quantity Held", "\t", "Price", "\t", "Classification")
        self._portfolio = dict(sorted(self._portfolio.items(), key = lambda item: item[1], reverse=True))
        for asset in self._portfolio:
            print(asset.name, "\t", self._portfolio[asset], "\t\t", asset.price, "\t", asset.classification)
                


class Asset():
    def __init__(self, name, quantity, price, classification):
        self.name = name
        self.quantity = quantity
        self.price = price
        self.classification = classification

    def __lt__(self, other):
        """Defines the behaviour of the < operator when dealing with members of the Asset class"""
        return self.name < other.name
        


class Exchange():
    def __init__(self, name, country):
        self.name = name
        self.country = country
        self._assets = ReadAssets("assets.csv").assets

    
    @property
    def country(self):
        return self._country


    @country.setter
    def country(self, country):
        """Setter method that validates and sets the user's country
        
        Reads in a list of countries and checks that the user's country is in this list
        If it is not, this implies that the user has provided a country that does not exist, and this will trigger an error
        """
        self._country_list = ReadCountries("countries.csv").countries
        if country in self._country_list:
            self._country = country

        else:
            raise ValueError("Error, invalid country selected")


    def show_assets(self):
        """Function that displays information about the assets available for purchase on the exchange"""
        print(f"Assets available on {self.name}")
        print("Name", "\t", "Quantity", "\t", "Price", "\t", "Classification")
        self._assets.sort()
        for asset in self._assets:
            print(asset.name, "\t", asset.quantity, "\t\t", asset.price, "\t", asset.classification)


    def process_buy(self, asset_requested, quantity_requested, user_balance):
        """Function that processes the user's buy request
        
        First checks that the requested asset exists
        Then checks that a sufficient quantity of the asset exists
        Then checks that the user has enough cash to pay for the asset
        If all of these criteria are met, the transaction is deemed to be valid
        For valid transactions, the quantity of available assets is reduced to allow for these transactions
        """
        for asset in self._assets:
            # Check that the asset exists
            if asset.name == asset_requested:

                # Check that a sufficient quantity of the asset exists
                if asset.quantity >= quantity_requested: 

                    # Check that the user has enough money to purchase the asset
                    if user_balance >= quantity_requested * asset.price:
                        asset.quantity -= quantity_requested
                        return True, asset

                    else:
                        print("Error, invalid funds")
                        print(f"Required funds: {quantity_requested * asset.price}")
                        print(f"Available funds: {user_balance}")
                        break

                else:
                    print("Error, insufficient quantity available")
                    print(f"Quantity requested: {quantity_requested}")
                    print(f"Quantity available: {asset.quantity}")
                    break

        else:
            print("Error, asset not found. Please check to ensure that the ticker symbol you have provided is correct")

        return False, None


    def process_sale(self, asset_ask, quantity_ask):
        """Function that processes the user's sell request
        
        Finds the sold asset in the available assets list and adds the quantity being sold to the asset
        """
        for asset in self._assets:
            if asset.name == asset_ask:
                asset.quantity += quantity_ask



class ReadAssets():
    def __init__(self, file):
        self.assets = {}
        self.file = file


    @property
    def assets(self):
        return ([x for x in self._assets])


    @assets.setter
    def assets(self, assets):
        self._assets = assets


    @property
    def file(self):
        return self._file


    @file.setter
    def file(self, file):
        """Setter method that ensures that the CSV file supplied is valid and exists"""
        if type(file) == str and os.path.isfile(file) and file.endswith(".csv"):
            self._file = file
            self._read_file()

        else:
            raise TypeError("Error, invalid input for file name")


    def _add_asset(self, asset):
        """Private method that splits each line in the CSV using comma as the delimiter, creates an Asset object, and puts this asset object in a dictionary"""
        asset = asset.split(",")
        # asset[0] = name, asset[1] = quantity, asset[2] = price, asset[3] = classification
        new_asset = Asset(asset[0], int(asset[1]), int(asset[2]), asset[3])
        self._assets[new_asset] = "Dval"


    def _read_file(self):
        """Private method for opening the file and iterating through the lines in the file
        
        The extracted lines represent assets and are provided to the private add_asset method
        """
        with open(self._file, "r") as fh:
            # Remove the first entry in readlines() to allow for the csv headers
            for line in fh.readlines()[1:]:
                self._add_asset(line[:-1])


    

class ReadCountries():
    def __init__(self, file):
        self.countries = {}
        self.file = file


    @property
    def countries(self):
        return ([x for x in self._countries])


    @countries.setter
    def countries(self, countries):
        self._countries = countries
        

    @property
    def file(self):
        return self._file


    @file.setter
    def file(self, file):
        """Setter method that ensures that the CSV file supplied is valid and exists"""
        if type(file) == str and os.path.isfile(file) and file.endswith(".csv"):
            self._file = file
            self._read_file()

        else:
            raise TypeError("Error, invalid input for file name")


    def _add_country(self, new_country):
        """Private method for adding countries to the country dictionary"""
        self._countries[new_country] = "Dval"


    def _read_file(self):
        """Private method for opening the file and iterating through the lines in the file
        
        The extracted lines represent countries which are provided to the _add_country method
        """
        with open(self._file, "r") as fh:
            for line in fh.readlines():
                self._add_country(line[:-1])



# Design

An online brokerage acts as an intermediary between users (traders) and their desired assets. Users buy and sell assets, and these transactions are validated by the exchange. Each user will have a name, country that they are registered in for tax purposes, a date of birth which should be over 18, a portfolio of assets, a cash balance, an exchange that they trade on, and a password. It is assumed that there is only one exchange that the user can trade on. Users specify the name of the assets and quantity that they wish to buy/sell. If a buy request is deemed to be valid by the exchange, the asset will be added to their portfolio (in the case where the asset already exists in the portfolio, the quantity held will increase) and their cash balance will be reduced. A similar operation is performed for the sell request. 

An exchange has a name, a country that it is registered in for tax purposes, and a list of available assets. These assets are read in through the use of a ReadAssets class, and this asset list is updated every time a user makes a valid transaction. When the exchange receives a buy request from the user, it checks that the asset exists, that a sufficient quantity of the asset exists, and that the user has sufficient cash to pay for the asset. If all of these criteria are met, the transaction is deemed to be valid, and the quantity of the asset available is reduced. A similar process occurs when the exchange receives a sell request.  

# Testing

## Create Exchange and show available assets

In [71]:
ex1 = Exchange("test exchange", "United States of America (USA)")
print(ex1.name)
print(ex1.country)
ex1.show_assets()

test exchange
United States of America (USA)
Assets available on test exchange
Name 	 Quantity 	 Price 	 Classification
AAPL 	 20000 		 300 	 stock
AMZN 	 25000 		 500 	 stock
FB 	 10000 		 200 	 stock
GME 	 30000 		 3000 	 stock
PLTR 	 5000 		 50 	 stock
SPY 	 50000 		 450 	 etf
TSLA 	 50000 		 1000 	 stock


## Create User

In [72]:
user1 = User("Michael Davitt", "Ireland, Republic of", "25/07/1998", "password", 500000, ex1)

## Perform valid transactions

In [73]:
user1.buy("SPY", 200)
user1.sell("SPY", 100)
user1.buy("TSLA", 20)

Successfully purchased 200 shares of SPY
Current balance: 410000
Successfully sold 100 shares of SPY
Current balance: 455000
Successfully purchased 20 shares of TSLA
Current balance: 435000


## Perform invalid transactions

In [74]:
user1.buy("AMD", 5000)
user1.buy("GME", 40000)
user1.buy("GME", 4000)
user1.sell("GME", 4000)

Error, asset not found. Please check to ensure that the ticker symbol you have provided is correct
Falied to purchase 5000 shares of AMD
Error, insufficient quantity available
Quantity requested: 40000
Quantity available: 30000
Falied to purchase 40000 shares of GME
Error, invalid funds
Required funds: 12000000
Available funds: 435000
Falied to purchase 4000 shares of GME
Error: GME not found in portfolio
Please ensure that you own the asset you wish to sell and that you have input the correct ticker symbol


## Show Portfolio

In [75]:
user1.show_portfolio()

Michael Davitt's Portfolio
Name 	 Quantity Held 	 Price 	 Classification
SPY 	 100 		 450 	 etf
TSLA 	 20 		 1000 	 stock


## Show Updated Asset Availability list

In [76]:
ex1.show_assets()

Assets available on test exchange
Name 	 Quantity 	 Price 	 Classification
AAPL 	 20000 		 300 	 stock
AMZN 	 25000 		 500 	 stock
FB 	 10000 		 200 	 stock
GME 	 30000 		 3000 	 stock
PLTR 	 5000 		 50 	 stock
SPY 	 49900 		 450 	 etf
TSLA 	 49980 		 1000 	 stock
